<div>
<img src="https://www.nebrija.com/images/logos/logotipo-universidad-nebrija.jpg" width="200">
</div>

**ALGORITMOS** -
Prof: Carmen Pellicer Lostao

# Optimizacion de Carteras o Portfolio


En este notebook vamos a utilizar el método de optimizacion de carteras de activos llamado MVO (Mean Variance Optimization) para calcular la composicion optima de una cartera. Resolveremos este ejercicio de forma clásica y cuantica, utilizando [algoritmos cuánticos variacionales](https://learning.quantum-computing.ibm.com/course/variational-algorithm-design) de optimizacion.

Dado un conjunto de $n$ activos $A_1, A_2,...,A_n$ podemos encontrar la composicion de una cartera optima expresada con un vector de pesos binario $x \in \{0, 1\}^n$, tal que maximice para el inversor los beneficios descontando riesgos (o que minimice la expresion opuesta).

## Método de optimizacion de carteras de activos llamado MVO (Mean Variance Optimization) 

El método MVO se basa en la hipotesis de que la distribucion de rentabilidades diarias de las acciones es normal y mide el beneficio como la media de esta distribucion y el riesgo como su varianza.

La rentabilidad y el riesgo de una cartera se modelan utilizando la *media* y la *varianza* de las fluctuaciones de los beneficios de los valores respectivamente.

Sea P una cartera de valores $A_1, A_2,...A_n$ con pesos binarios $x_1, x_2,...x_n$ y las rentabilidades $\mu_1, \mu_2, ...\mu_n$.

La **rentabilidad de la cartera $\mu$**  viene determinada por la suma de las medias ponderada de las rentabilidades diarias $\mu_i$de los valores que la componen:

$\mu= \mu^T x = \Sigma_{i=1}^{n} x_i * \mu_i$

El **riesgo de la cartera $\sigma$** es la desviacion estandar de la rentabilidad, que podemos calcular a partir de la matriz de covarianzas:

$\sigma = x^T \sigma x = \sqrt{\Sigma_{i=1}^{n} \Sigma_{j=1}^{n} x_i x_j \sigma_{i,j}}$

Por tanto, el método MVO propone que una cartera optima será aquella que cumpla la siguiente ecuacion:

$$\begin{aligned}
\min_{x \in \{0, 1\}^n}  q x^T \sigma x - \mu^T x\\
\text{sujeto a que: } 1^T x = B
\end{aligned}$$

donde se tiene:

- $x \in \{0, 1\}^n$ es un vector binario que indica si un valor esta o no en la cartera. Le llamamos el vector de decision, que indica que la acción $i$ esta en la cartera si $x[i] = 1$ y no esta si $x[i] = 0$,
- $\mu \in \mathbb{R}^n$ son las medias de las rentabilidades diarias de las acciones,
- $\sigma \in \mathbb{R}^{n \times n}$ son las covarianzas entre las rentabilidades de las acciones, lo que varia una si varia otra,
- $q > 0$ es un parametro con el que codificamos la afinidad al riesgo del inversor que diseña la cartera,
- y $B$ es el presupuesto o 'budget', i.e. limita el numero de acciones que podemos seleccionar sin superarlo.

La restriccion de que $1^T x = B$ se codifica como un termino de penalizacion $(1^T x - B)^2$ escalado por un parámetro y que se resta a la función objetivo o funcion de coste que vamos a minimizar. 

El problema resultante, es un problema QUBO (Quadratic Unconstrained Binary Optimization) y puede mapearse a una funcion de coste cuantica o Hamiltoniano cuyo estado minimo es la solucion optima.

Utilizaremos dos metodos de optimizacion cuántica para resolverlo:
- El Variational Quantum Eigensolver (VQE) 
- El Quantum Approximate Optimization Algorithm (QAOA)

Instalamos la librería de algoritmos e importamos las librerias necesarias

In [ ]:
#!pip install qiskit-algorithms

In [ ]:
#!pip install qiskit_finance

In [ ]:
from qiskit import Aer, transpile
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance
from qiskit.providers.fake_provider import FakeVigo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.applications import OptimizationApplication
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_finance.applications.optimization import PortfolioOptimization
from qiskit.utils import algorithm_globals
from qiskit.quantum_info import Statevector
from qiskit.tools.visualization import plot_histogram, plot_state_city

from qiskit.primitives import Estimator  #RUNTIME PRIMITIVE ESTIMATOR
from qiskit.circuit.library import TwoLocal

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import VQE, QAOA, SamplingVQE, NumPyMinimumEigensolver  # new import!!!
from qiskit_algorithms import NumPyMinimumEigensolver

from qiskit.algorithms.optimizers import SPSA, COBYLA, L_BFGS_B, SLSQP
from qiskit.primitives import Estimator, Sampler
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler, Estimator

### Seleccion de valores para la cartera

Utilizamos las librerias de Yahoo Finance para obtener un historico de cotizaciones de los valores que queramos utilizar en el ejercicio.

Algunos tickers de acciones que podriamos utilizar para el ejercicio podrían ser:

- valores o tickers Dow Jones:

['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'UTX', 'V', 'VZ', 'WBA', 'WMT', 'XOM']

- valores o tickers España Mercado Continuo podrían ser: 

['FDR.MC','AMS.MC','PHM.MC','IBE.MC','REE.MC','ENG.MC','CABK.MC','GRF.MC','SAB.MC','ACS.MC','BBVA.MC','MTS.MC','ELE.MC','FER.MC','CLNX.MC','BKT.MC','TEF.MC','ACX.MC','ITX.MC','MAP.MC','AENA.MC','SAN.MC','REP.MC','SGRE.MC','NTGY.MC','MEL.MC','IAG.MC','MRL.MC','COL.MC','ANA.MC']


Instalamos las librerias

In [ ]:
#!pip install yfinance

In [ ]:
#!pip install yfinance
import yfinance as yfin

In [ ]:
#Podemos obtner datos de valores de las cotizaciones bursatiles de Yahoo que usamos para nuestro proyecto
tickers=['BBVA.MC', 'ACX.MC', 'NTGY.MC', 'REP.MC', 'CABK.MC']  #tomamos solo 5 valores, cada valor va a ser una variable binaria y requiere de 1 qubit
num_assets=len(tickers)

df_stocks = yfin.download(tickers=tickers, start='2023-01-01',end='2024-02-01')
df_stocks_aclose=df_stocks['Adj Close']
df_stocks_aclose.head()

Preparamos los datos comprobando que no hay nulos y si los hay completamos los valores con las cotizaciones pasadas mas cercanas. 

Para ello transformamos un `DataFrame` para que los valores de cotizaciones que no vienen informados, tomen la cotizacion del dia anterior.

Utilizamos los metodos `.fillna()` con valor `ffill` **fill forward** y luego `bfill ` **fill backwards** para completar los datos de cotizaciones sin introducir informacion adicional

In [ ]:
no_info=df_stocks_aclose.isnull().sum().sum()

if  no_info != 0:
    print('Hay datos no informados')
    #ffill se debe de ejecutar primero para no meter datos del futuro (por si hacemos prediccion)
    #rellena hacia abajo, hacia el futuro - si el primer dato del pasado esta vacio no se podra llenar
    #se ejecuta entonces despues bfill para rellenar ese valor
    df_stocks_aclose=df_stocks_aclose.fillna(method='ffill').fillna(method='bfill').head(10) #el priner metodo que se ejecuta es el mas interno y a su resultado ejecuta el externo
        
else:
    print('Todos los datos estan informados')

Creamos el vector de medias de beneficios y la matriz de covarianzas o riesgos

In [ ]:
stockReturns=pd.DataFrame()
#computamos la rentabilidad diaria de cada accion
stockReturns= (df_stocks_aclose/df_stocks_aclose.shift() - 1)* 100
stockReturns.head()

In [ ]:
#quitamos la primera fila
stockReturns.drop(labels=stockReturns.index[0], axis=0, inplace=True)
stockReturns

In [ ]:
mu=stockReturns.mean().to_numpy()
print('Vector de valores medios de los beneficios diarios de las acciones:',mu)

In [ ]:
stockReturns.hist(figsize=(10,10));

In [ ]:
#computamos la matriz de covarianzas de las rentabilidades 
sigma = stockReturns.cov().to_numpy()
print('Matriz de varianza covarianza de las acciones:\n', sigma)

In [ ]:
# plot sigma
plt.imshow(sigma, interpolation="nearest")
plt.colorbar()
plt.xticks(range(num_assets))
plt.yticks(range(num_assets))
plt.show()

## Definicion de la funcion de coste

A partir de la funcion objetivo de la optimizacion de la cartera, construimos una clase que la expresa como un problema tipo CUBO a partir de la cual podemos construir el circuito de coste para la optimizacion.

En Qiskit, exite una clase [`PortfolioOptimization`](https://qiskit-community.github.io/qiskit-finance/stubs/qiskit_finance.applications.PortfolioOptimization.html#) que nos permite escribir de una forma muy sencilla la funcion objetivo de la optimizacion de la cartera.

Esta clase contiene el metodo `to_quadratic_program()` que permite formular la ecuacion como un operador hamiltoniano y construir de forma mas o menos inmediata el circuito para computar el coste en los pasos de optimizacion.

#### EJERCICIO

Obten el objeto `QuadraticProgram` de nuestro problema de optimizacion MVO y su formulacion como objetos QUBO y operador Ising.

In [ ]:
num_assets = len(tickers)
q = 0.5  # fijamos un factor de riesgo para el diseño de la cartera optima
budget = num_assets // 2  # fijamos un budget
penalty = num_assets  # fijamos un parametro de penalizacion para incorporar la restriccion en el problema QUBO

portfolio = PortfolioOptimization(
    expected_returns=mu, covariances=sigma, risk_factor=q, budget=budget
)

qp = #tu codigo
print(qp.prettyprint())

In [1]:
#Para ver todos las propiedades y metodos de la clase PortfolioOptimization convertida a quadratic program
#dir(qp)

In [ ]:
#converting to QUBO
qubo = 
print("QUBO:")
print(str(qubo)+'\n')

#converting QUBO task to Ising Hamiltonian for simulation on quantum computer
ising, offset = 
print("ISING Hamiltonian:")
print(ising)
print(offset)

## Optimizacion clásica

Vamos a resolver el problema de forma clásica para poder contrartar los resultados con la forma cuántica y ver que se optiene la misma solucion.

Podemos hacer una optimizacion clásica utilizando las librerias numericas de python `numpy`. Esto esta ya programado en Qiskit gracias a la clase `NumPyMinimumEigensolver` a la que podemos pasar una funcion de coste en forma de programa cuadrático y nos devuelve la solucion optima con los datos en un diccionario.

Esta clase, toma el operador hamiltoniano de la funcion de coste y calcula el valor propio minimo de forma numerica clasica.

#### EJERCICIO

Utiliza el método `compute_minimum_eigenvalue` de la clase [`NumpyMinimumEigensolver`](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.NumPyEigensolver.html) para resolver la optimizacion del operador H (el problema formulado como Ising el vector propio encontrando el vector propio de minima energia.

Convierte en un objeto tipo `Statevector` el resultado e imprime el diccionario de probabilidades que tiene con el metodo `probabilities_dict()`

In [ ]:
#solve the optimization problem

#create Statevector with the eigenvector or eigenstate and print the probabilities dictionary


## Optimizacion cuántica utilizando el algoritmo VQE

Utilizaremos el algoritmo Variational Quantum Eigensolver (VQE) para resolver el problema de optimizacion de la cartera.

Para ello necesitamos unos pasos previos:

1) construir un circuito variacional o `ansatz` con unos parametros. Utilizaremos la clase [TwoLocal](https://qiskit.org/documentation/stubs/qiskit.circuit.library.TwoLocal.html) de Qiskit que construye un circuito variacional a conveniencia, con diferentes parametros y configuraciones.

2) Seleccionar una funcion de optimizacion clasica que nos guia para encontrar el minimo.

3) Definimos un backend de ejecucion, un simulador

Después usaremos la clase `VQE` de Qiskit para definir el optimizador y le pasaremos el ansatz, la funcion de optimizacion y el backend de ejecucion.

Utilizamos la clase `MinimumEigenOptimizer` para lanzar el proceso de optimizacion y le pasaremos la funcion de coste cuadratica del metodo MVO para que evalue los costes y encuentre el minimo

#### EJERCICIO

Utiliza el método `compute_minimum_eigenvalue()` de la clase [`SamplingVQE`](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.SamplingVQE.html#qiskit_algorithms.SamplingVQE) para resolver la optimizacion del operador H (el problema formulado como Ising), encontrando el vector propio de minima energia.

Investiga la clase de resultado que devuelve este algoritmo [SamplingVQEResult](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.SamplingVQEResult.html) y utilizando el metodo `best_measuretments()` imprime el resultado binario y dibuja el grafo obteindo de la minimizacion.

In [ ]:
n=num_assets


#### EJERCICIO

Muestra el circuito variacional optimo que ha encontrado el optimizador, ejecuta el circuito y muestra el histograma de salida, para ver el estado optimo que hemos encontrado

In [ ]:
#podemos ver el circuito optimo


#podemos ejecutarlo, uno de esos estados que ha muestreado el circuito variacional ha dado minima energia


# Hacemos un transpile del circuito para el simulador
simulator = Aer.get_backend('aer_simulator')
circ = transpile(circuit, simulator)  #opcional

# lo corremos y obtenemos los resultados

## Optimizacion cuántica utilizando el algoritmo QAOA

Utilizaremos el algoritmo Qantum Approximate Optimization Algorithm (QAOA) para resolver el problema de optimizacion de la cartera. Este es otro algoritmo que utiliza un circuito variacional definido internamente y no necesitamos crearlo explicitamente.

Para ejecutarlo necesitamos los pasos previos anteriores:

1) Seleccionar una funcion de optimizacion clasica que nos guia para encontrar el minimo.

2) Definimos un backend de ejecucion, un simulador

Después usaremos la clase `QAOA` de Qiskit para definir el optimizador y le pasaremos el ansatz, la funcion de optimizacion y el backend de ejecucion. Tambien tendremos que especificar el numero de repeticiones que ejecutaremos en el algoritmo `QAOA`.

Utilizamos la clase `MinimumEigenOptimizer` para lanzar el proceso de optimizacion y le pasaremos la funcion de coste cuadratica del metodo MVO para que evalue los costes y encuentre el minimo

#### EJERCICIO

Utiliza el método `compute_minimum_eigenvalue()` de la clase [`QAOA`](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.QAOA.html) para resolver la optimizacion del operador H (el problema formulado como Ising), encontrando el vector propio de minima energia.

La clase `QAOA` se contruye a partir de la clase `SamplingVQE`por lo que devuelve la misma clase de resultados, [SamplingVQEResult](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.SamplingVQEResult.html). Utiliza el metodo `best_measuretments()` imprime el resultado binario y dibuja el grafo obteindo de la minimizacion.

In [2]:
#resuelve el problema de optimizacion con el algoritmo QAOA


#### EJERCICIO

Muestra el circuito variacional optimo que ha encontrado el optimizador, ejecuta el circuito y muestra el histograma de salida, para ver el estado optimo que hemos encontrado

In [ ]:
#podemos ver el circuito optimo


#podemos ejecutarlo, uno de esos estados que ha muestreado el circuito variacional ha dado minima energia


# Hacemos un transpile del circuito para el simulador
simulator = Aer.get_backend('aer_simulator')
circ = transpile(circuit, simulator)  #opcional

# lo corremos y obtenemos los resultados
